In [26]:
import pickle

import pandas as pd

In [27]:
# データの読み込み
df = pd.read_csv("./data/data_without_nan.tsv", sep="\t")
df

,ラベル,満足度,文章,会員ID
0,医療・福祉,1.0,迅速,1231234771
1,買物・飲食,0.5,便利です,1231234771
2,移動・交通,-0.5,便利,1231234771
3,遊び・娯楽,-1.0,普通,1231234771
4,移動・交通,1.0,鉄道路線が充実している。どこにいくにも便利。,1226930387
...,...,...,...,...
1968,遊び・娯楽,-1.0,娯楽の場所がない,1234360589
1969,移動・交通,1.0,通っている路線が多い,1235895537
1970,買物・飲食,0.5,駅前にスーパー、ドラッグストア、100均、飲食店など生活に必要な施設が揃っている。,1235895537
1971,事故・犯罪,-0.5,ストーカーが多い。変質者が多い。,1235895537


In [28]:
# ある長さ以上の文章のみを抽出し、バッチサイズごとにデータを分割して保存する

# 最低限の文字列の長さを設定
min_length = 20
# バッチサイズを設定
batch_size = 64

# "文章"列の中身の長さがmin_length以上のデータを抽出
long_texts_df = df[df["文章"].str.len() >= min_length].copy()

# "満足度"列の値を変換
long_texts_df.loc[:, "満足度"] = long_texts_df["満足度"].apply(
    lambda x: -1 if x == -0.5 else 1 if x == 0.5 else x
)

# # "満足度"の列の値が1なら"満足"、-1なら"不満"としてラベルを付与
# long_texts_df.loc[:, "満足度"] = long_texts_df["満足度"].apply(
#     lambda x: "満足" if x == 1 else "不満"
# )

# バッチサイズごとにデータを分割
df_batches = [
    long_texts_df[i : i + batch_size] for i in range(0, len(long_texts_df), batch_size)
]

print(f"元のデータの行数: {len(df)}")
print(f"{min_length}文字以上のデータの行数: {len(long_texts_df)}")

元のデータの行数: 1973
20文字以上のデータの行数: 731


In [29]:
long_texts_df
# df_batches[0]

,ラベル,満足度,文章,会員ID
4,移動・交通,1.0,鉄道路線が充実している。どこにいくにも便利。,1226930387
8,自然景観,1.0,海も近く県立公園もあり、リラックスできるので。,1233151099
12,移動・交通,1.0,今居住している武蔵小杉はJR横須賀線、南武線、東急東横線、目黒線と選択の余地がたくさんあり、...,1229140632
13,買物・飲食,1.0,スーパーマーケットもたくさんあり、選択の余地が広い,1229140632
14,遊び・娯楽,-1.0,かなり発展した街なのに映画館がない。ライブハウスがない。,1229140632
...,...,...,...,...
1941,デジタル生活,-1.0,行政関係の申請や交付をするときに、市役所まで出向いて行う必要があり、デジタル行政関係が遅れて...,1236838426
1942,医療・福祉,-1.0,高齢者福祉・支援や障害者支援などを包括的に対応できる具体的な体制や組織が不十分であると感じている。,1236838426
1945,医療・福祉,-1.0,南部地区に総合病院がないので設置してほしい。,1231797204
1946,買物・飲食,-1.0,藤沢駅周辺のショッピング環境を更に改善してほしい。,1231797204


In [30]:
# 保存
with open("./data/df_batches.pkl", "wb") as f:
    pickle.dump(df_batches, f)

In [31]:
# 正解データを辞書型で保存

long_texts_df["満足度"] = long_texts_df["満足度"].astype(str)
# "満足度"の列の値が1なら"満足"、-1なら"不満"としてラベルを付与
long_texts_df.loc[:, "満足度"] = (
    long_texts_df["満足度"].apply(lambda x: "満足" if x == "1.0" else "不満")
)
original_data_dict = {
    index: {"label": row["ラベル"], "sentiment": row["満足度"], "text": row["文章"]}
    for index, row in long_texts_df.iterrows()
}
# original_data_dict

In [32]:
# 保存
with open("./data/original_data_dict.pkl", "wb") as f:
    pickle.dump(original_data_dict, f)